In [ ]:
import math

def gives_angle(perp_distance, eyes_distance):
    hypotenuse = math.sqrt(perp_distance**2 + eyes_distance**2)
    angle = math.atan(perp_distance/eyes_distance)
    return angle

    


In [15]:
import cv2
import  math
# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


# Initialize the video capture
cap = cv2.VideoCapture(0)
eye_coordinates = []


REFERENCE_OBJECT_WIDTH = 20  # Width of the reference object in centimeters
FOCAL_LENGTH = 550  # Focal length of the camera in pixels

def calculate_distance_to_object(object_width, focal_length, measured_width):
    # Calculate the distance to the object using the formula: distance = (reference_width * focal_length) / measured_width
    distance = (object_width * focal_length) / measured_width
    return distance

def calculate_angle_change(x1, y1, x2, y2):
    # Calculate the angle between two sets of eye coordinates
    dx = x2 - x1
    dy = y2 - y1
    angle_radians = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees


screen_width = 1920
screen_height = 1080

distance = 0

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,10)
    for (x,y,w,h) in faces:
        distance = calculate_distance_to_object(REFERENCE_OBJECT_WIDTH, FOCAL_LENGTH, w)
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
    blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)
    eyes = eye_cascade.detectMultiScale(blurred_gray, scaleFactor=1.3, minNeighbors=10)
    eye_coordinates = []
    
        

    if len(eyes) >= 2:

        # Iterate over the detected eyes
        for (ex, ey, ew, eh) in eyes[:2]:
            # Extract the eye region from the frame
            eye = frame[ey:ey+eh, ex:ex+ew]

            # Convert the eye region to grayscale
            eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(eye_gray, 30, 255, cv2.THRESH_BINARY)

            # Find contours in the binary image
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Iterate over the contours
            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Filter contours based on area to find the dark portion of the eyeball
                if area > 100:
                    # Find the centroid of the contour using the moments
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])

                        # Draw a red dot at the center of the dark portion of the eyeball
                        cv2.circle(eye, (cx, cy), 2, (0, 0, 255), -1)
                
                        


            # Append the coordinates of the red dot to the list
            eye_coordinates.append([ex+cx, ey+cy])
            
            # Replace the eye region back in the frame
            frame[ey:ey+eh, ex:ex+ew] = eye
            
            # frame = cv2.circle(frame, (int(ex+ew/2), int(ey+eh/2)), 5, (0, 255, 0), -1)

        


        eye_coordinates = sorted(eye_coordinates, key=lambda x: x[0])

        left_eye_center_x = eye_coordinates[0][0]
        right_eye_center_x = eye_coordinates[1][0]
        left_eye_center_y = eye_coordinates[0][1]
        right_eye_center_y = eye_coordinates[1][1]
        
        # angle = calculate_angle_change(left_eye_center_x, left_eye_center_x, left_eye_center_x, right_eye_center_y)


        
        



        
        # print(angle)


        
        
            
        # print(eye_coordinates)
        
        
        # print(eye_coordinates)
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2

# Constants for reference object size and distance
REFERENCE_OBJECT_WIDTH = 20  # Width of the reference object in centimeters
FOCAL_LENGTH = 550  # Focal length of the camera in pixels

def calculate_distance_to_object(object_width, focal_length, measured_width):
    # Calculate the distance to the object using the formula: distance = (reference_width * focal_length) / measured_width
    distance = (object_width * focal_length) / measured_width
    return distance

# Start the webcam
cap = cv2.VideoCapture(0)

# Load the face cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 10)

    # Iterate over detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        
        # Calculate the distance to the person's face
        distance = calculate_distance_to_object(REFERENCE_OBJECT_WIDTH, FOCAL_LENGTH, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Distance Estimation', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2

# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    if ret:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect eyes in the blurred grayscale frame
        eyes = eye_cascade.detectMultiScale(blurred_gray, scaleFactor=1.3, minNeighbors=10)

        # Check if both eyes are detected
        if len(eyes) == 2:
            # Extract the eye centers and sort them based on x-coordinate and y-coordinate
            eye_centers = sorted([(ex + ew // 2, ey + eh // 2) for (ex, ey, ew, eh) in eyes])

            left_eye_center = eye_centers[0]
            right_eye_center = eye_centers[1]

            # Compare the relative positions of eye centers to determine the gaze direction
            horizontal_gaze = "Right" if right_eye_center[0] > left_eye_center[0] else "Left"
            vertical_gaze = "Up" if right_eye_center[1] < left_eye_center[1] else "Down"

            # Draw circles around the eye centers on the frame
            cv2.circle(frame, left_eye_center, 2, (0, 0, 255), -1)
            cv2.circle(frame, right_eye_center, 2, (0, 0, 255), -1)

            # Display the gaze direction on the frame
            cv2.putText(frame, f"Horizontal Gaze: {horizontal_gaze}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Vertical Gaze: {vertical_gaze}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


In [7]:
import tensorflow as tf
import numpy as np
import PIL.Image

# Load pre-trained VGG19 model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

# Extract specific layers from VGG19 for style representation
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
style_outputs = [vgg.get_layer(name).output for name in style_layers]

# Build the model
model = tf.keras.Model(inputs=vgg.input, outputs=style_outputs)
model.trainable = False

# Preprocess images by resizing and normalizing pixel values
def preprocess_image(image):
    image = tf.keras.applications.vgg19.preprocess_input(image)
    image = tf.image.resize(image, (224, 224))
    return image

# Convert tensor to image
def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# Load content and style images
content_image = PIL.Image.open('1.jpg')
style_image = PIL.Image.open('2.jpg')

# Preprocess content and style images
content_array = np.array(content_image)
content_array = np.expand_dims(content_array, axis=0)
content_tensor = tf.constant(preprocess_image(content_array))

style_array = np.array(style_image)
style_array = np.expand_dims(style_array, axis=0)
style_tensor = tf.constant(preprocess_image(style_array))

# Compute style features
style_features = model(style_tensor)

# Define content and style weights
content_weight = 1e3
style_weight = 1e-2

# Initialize generated image as the content image
generated_image = tf.Variable(content_tensor, dtype=tf.float32)

# Define loss function for style transfer
def style_content_loss(style_targets, content_targets, generated_targets):
    style_loss = tf.add_n([tf.reduce_mean((style_target - tf.image.extract_patches(generated_target, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='VALID')))**2
                           for style_target, generated_target in zip(style_targets, generated_targets)])

    content_loss = tf.reduce_mean((content_targets - generated_targets)**2)

    total_loss = content_weight * content_loss + style_weight * style_loss
    return total_loss

# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

# Perform style transfer iterations
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        style_features_gen = model(image)
        loss = style_content_loss(style_features, content_tensor, style_features_gen)
    gradients = tape.gradient(loss, image)
    optimizer.apply_gradients([(gradients, image)])
    image.assign(tf.clip_by_value(image, clip_value_min=-1.0, clip_value_max=1.0))

num_iterations = 1000

for i in range(num_iterations):
    train_step(generated_image)

# Convert final generated image tensor to image
stylized_image = tensor_to_image(generated_image.numpy())

# Save the stylized image
stylized_image.save('stylized_image.jpg')


ValueError: in user code:

    File "C:\Users\baral\AppData\Local\Temp\ipykernel_11228\2485610761.py", line 72, in train_step  *
        loss = style_content_loss(style_features, content_tensor, style_features_gen)
    File "C:\Users\baral\AppData\Local\Temp\ipykernel_11228\2485610761.py", line 56, in style_content_loss  *
        style_loss = tf.add_n([tf.reduce_mean((style_target - tf.image.extract_patches(generated_target, sizes=[1, 3, 3, 1], strides=[1, 1, 1, 1], rates=[1, 1, 1, 1], padding='VALID')))**2

    ValueError: Dimensions must be equal, but are 224 and 222 for '{{node sub}} = Sub[T=DT_FLOAT](sub/x, ExtractImagePatches)' with input shapes: [1,224,224,64], [1,222,222,576].
